In [1]:
import os
import random
import time

In [10]:
USER_DATA_FILE = "user_data.txt"
SCORES_FILE = "scores.txt"

In [14]:
def load_data(file_path):
    if not os.path.exists(file_path):
        return {}
    data = {}
    with open(file_path, "r") as file:
        for line in file:
            try:
                key, value = line.strip().split(":", 1)
                data[key] = eval(value) if file_path == USER_DATA_FILE else int(value)
            except ValueError:
                print(f"Skipping malformed line in {file_path}: {line.strip()}")
    return data


In [15]:
def save_data(file_path, data):
    with open(file_path, "w") as file:
        for key, value in data.items():
            file.write(f"{key}:{value}\n")

In [16]:
user_data = load_data(USER_DATA_FILE)
scores = load_data(SCORES_FILE)
admin_credentials = {"username": "admin", "password": "admin"}

In [17]:
def register_user():
    user_id = len(user_data) + 1
    username = input("Enter a new username: ")
    if username in user_data:
        print("Username already exists! Please try logging in.")
        return None
    password = input("Enter a new password: ")
    full_name = input("Enter your full name: ")
    email = input("Enter your email: ")
    phone = input("Enter your phone number: ")

    user_data[username] = {
        "id": user_id,
        "password": password,
        "full_name": full_name,
        "email": email,
        "phone": phone,
    }
    scores[username] = 0
    save_data(USER_DATA_FILE, user_data)
    save_data(SCORES_FILE, scores)
    print("Registration successful! Please log in.")


In [26]:
def login_user():
    username = input("Enter username: ")
    password = input("Enter password: ")
    if username in user_data:
        if user_data[username]["password"] == password:
            print(f"Welcome, {username}!")
            return username
        else:
            print("Invalid password! Please try again.")
    else:
        print("Username not found! Please register first.")
    return None

In [19]:
def admin_operations():
    print("\nAdmin Panel")
    while True:
        print(
            "1. View All Users Information\n2. View Leaderboard\n3. Edit User Information\n4. Logout"
        )
        choice = input("Select an option: ")
        if choice == "1":
            print("\nAll Users Information:")
            for username, details in user_data.items():
                print(
                    f"ID: {details['id']}, Username: {username}, Full Name: {details['full_name']}, Email: {details['email']}, Phone: {details['phone']}"
                )
        elif choice == "2":
            print("\nLeaderboard:")
            sorted_scores = sorted(scores.items(), key=lambda x: x[1], reverse=True)
            for user, score in sorted_scores:
                print(f"Username: {user}, Score: {score}")
        elif choice == "3":
            user_id_to_edit = input("Enter the user ID to edit: ")
            for username, details in user_data.items():
                if str(details["id"]) == user_id_to_edit:
                    full_name = (
                        input(
                            f"Enter new full name (current: {details['full_name']}): "
                        )
                        or details["full_name"]
                    )
                    email = (
                        input(f"Enter new email (current: {details['email']}): ")
                        or details["email"]
                    )
                    phone = (
                        input(f"Enter new phone (current: {details['phone']}): ")
                        or details["phone"]
                    )

                    user_data[username]["full_name"] = full_name
                    user_data[username]["email"] = email
                    user_data[username]["phone"] = phone

                    save_data(USER_DATA_FILE, user_data)
                    print("User information updated successfully.")
                    break
            else:
                print("User ID not found!")
        elif choice == "4":
            print("Logging out from Admin Panel...\n")
            break
        else:
            print("Invalid choice. Try again.")


In [20]:
WORDS_WITHOUT_VOWELS = {
    "apple": "ppl", "banana": "bnn", "cherry": "chrry", "date": "dt",
    "grape": "grp", "kiwi": "kw", "lime": "lm", "mango": "mng",
    "melon": "mln", "orange": "rng", "peach": "pch", "pear": "pr",
    "plum": "plm", "berry": "brry", "papaya": "ppy", "fig": "fg",
    "apricot": "prct", "guava": "gv", "pineapple": "pnpple", "durian": "drn",
    "jackfruit": "jckfrt", "nectarine": "nctrn", "persimmon": "prsmn",
    "pomegranate": "pmgrnt", "watermelon": "wtrmln", "avocado": "vcd",
    "coconut": "ccnt", "blueberry": "blbrry", "raspberry": "rspbrry",
    "blackberry": "blckbrry", "lychee": "lych", "elderberry": "ldrbrry",
    "starfruit": "strfrt", "grapefruit": "grpfrt", "lemon": "lmn"
}

In [21]:
def play_game(username):
    print("\nWelcome to the Missing Vowels Game!")
    words = random.sample(list(WORDS_WITHOUT_VOWELS.keys()), 5)
    correct_guesses = 0
    start_time = time.time()

    for word in words:
        elapsed_time = time.time() - start_time
        remaining_time = 120 - elapsed_time
        if remaining_time <= 0:
            print("\nTime's up! GAME OVER.")
            break

        print(f"\nTime remaining: {int(remaining_time)} seconds")
        scrambled = WORDS_WITHOUT_VOWELS[word]
        print(f"Word with missing vowels: {scrambled}")
        guess = input("Guess the word: ").strip().lower()

        if guess == word:
            print("Correct!")
            correct_guesses += 1
        else:
            print(f"Incorrect! The correct word was: {word}")

    scores[username] += correct_guesses
    save_data(SCORES_FILE, scores)
    print(f"\nYour total score: {scores[username]}\n")

In [22]:
def user_home(username):
    while True:
        print("\nHome Page")
        print("1. Play Missing Vowels Game\n2. View Score\n3. Logout")
        choice = input("Select an option: ")
        if choice == "1":
            play_game(username)
        elif choice == "2":
            print(f"\nYour total score: {scores[username]}\n")
        elif choice == "3":
            print("Logging out...\n")
            break
        else:
            print("Invalid choice. Try again.")

In [23]:
def main():
    print("Welcome to the Missing Vowels System!")
    while True:
        print("\n1. Login\n2. Register\n3. Admin Login\n4. Exit")
        choice = input("Select an option: ")
        if choice == "1":
            username = login_user()
            if username:
                user_home(username)
        elif choice == "2":
            register_user()
        elif choice == "3":
            admin_username = input("Enter admin username: ")
            admin_password = input("Enter admin password: ")
            if (
                admin_username == admin_credentials["username"]
                and admin_password == admin_credentials["password"]
            ):
                admin_operations()
            else:
                print("Invalid admin credentials!")
        elif choice == "4":
            print("Exiting the system. Goodbye!")
            break
        else:
            print("Invalid choice. Try again.")


In [ ]:
if __name__ == "__main__":
    main()

Welcome to the Missing Vowels System!

1. Login
2. Register
3. Admin Login
4. Exit
Select an option: 1
Enter username: ali
Enter password: 1111
Welcome, ali!

Home Page
1. Play Missing Vowels Game
2. View Score
3. Logout
Select an option: 1

Welcome to the Missing Vowels Game!

Time remaining: 120 seconds
Word with missing vowels: fg
Guess the word: eds
Incorrect! The correct word was: fig

Time remaining: 115 seconds
Word with missing vowels: strfrt
Guess the word: asdsa
Incorrect! The correct word was: starfruit

Time remaining: 107 seconds
Word with missing vowels: gv
Guess the word: guava
Correct!

Time remaining: 103 seconds
Word with missing vowels: chrry
Guess the word: cherry
Correct!

Time remaining: 99 seconds
Word with missing vowels: lych
Guess the word: lychee
Correct!

Your total score: 3


Home Page
1. Play Missing Vowels Game
2. View Score
3. Logout
Select an option: 2

Your total score: 3


Home Page
1. Play Missing Vowels Game
2. View Score
3. Logout
Select an option: